In [1]:
import sys
import json

params = {'general':{'period':'ip'}}
default = {'training_folder': '../..', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../..', 'params': {'general': {'period': 'ip'}}}


In [2]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [3]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
import numpy as np
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [4]:
period = argv['params']['general']['period']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if on_lambda:
    scenario_folder = training_folder +r'/inputs/'
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
else:
    scenario_folder = training_folder + '/scenarios/' + period + '/inputs/'
    output_folder = training_folder + '/scenarios/' + period + '/outputs/'
    model_folder = training_folder + '/scenarios/' + period + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)


pt_folder = scenario_folder + 'pt/'
road_folder = scenario_folder + 'road/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/ip/outputs/
scen folder :  ../../scenarios/ip/inputs/
model folder :  ../../scenarios/ip/model/


In [5]:
var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# inputs

In [6]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

road_nodes: 100%|███████████████████████████████| 25/25 [00:00<00:00, 39.58it/s]


volumes

In [7]:
filepath = excel.get_filepath(input_folder + '{s}/volumes/volumes.csv', ancestry=ancestry, log=False)
volumes = pd.read_csv(filepath)
volumes = set_index(volumes)

In [8]:
period_dict={'am':'AM','pm':'PM','ip':'IP','op':'OP'}
volumes = volumes[volumes['time_period'] == period_dict[period]]

In [9]:
volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

congested road time

In [10]:
road_times = pd.read_csv(road_folder+'road_times.csv')
road_times = set_index(road_times)
road_times['old_index'] = road_times.index
road_times.index = 'rlink_' + road_times.index.astype(str)

# apply traffic

In [11]:
road_times['time_ff'] = sm.road_links['time']

In [12]:
time_dict = road_times.set_index('old_index')['time_ff'].to_dict()
sm.links['time_ff'] = sm.links['road_link_list'].apply(lambda ls: sum([*map(time_dict.get, ls)]))


time_dict = road_times.set_index('old_index')['time'].to_dict()
sm.links['time_congestion'] = sm.links['road_link_list'].apply(lambda ls: sum([*map(time_dict.get, ls)]))

In [13]:
#sm.links['time'] = sm.links['time'] / sm.links['jam_factor'] 
sm.links['jam_factor'] = sm.links['time_ff'] / sm.links['time_congestion']

In [14]:
sm.links['jam_factor'] = sm.links['jam_factor'].apply(lambda x: max(x,0.1)).apply(lambda x: min(x,1.2))

In [15]:
# broken mode, pour avoir sans rail et sans bus.mes deux segment
ref = sm.copy()

In [16]:
sm.links['time'] = sm.links['time'] / sm.links['jam_factor'] 


# pathfinder

In [17]:
volumes_pt = volumes[volumes['vehicle_class'].isin(['RPAX','BPAX'])]
od_set = set(zip(volumes_pt['origin'], volumes_pt['destination']))


In [33]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster!
)


start publicpathfinder
build_mode_combinations
find_broken_mode_paths


breaking modes: set() : 100%|█████████████████████| 4/4 [00:24<00:00,  6.06s/it]


path_analysis


In [34]:
def format_los(sm,volumes_pt):
    sm.pt_los['broken_modes'] = sm.pt_los['broken_modes'].apply(str)
    time_per_modes = sm.pt_los.groupby(['origin','destination','broken_modes'])['gtime'].agg(sum).unstack().reset_index()

    los = volumes_pt[['origin','destination','vehicle_class']].copy()
    los = los.drop_duplicates()

    los = los.merge(time_per_modes,on = ['origin','destination'])
    #los = los.merge(best_paths[['origin','destination','gtime']],on = ['origin','destination'])
    
    los['time']= np.nan
    idx = los['vehicle_class']=='BPAX'
    los.loc[idx,'time'] = los.loc[idx,"{'rail'}"]

    idx = los['vehicle_class']=='RPAX'
    los.loc[idx,'time'] = los.loc[idx,"{'bus'}"]

    los['all_walk'] = False
    los.loc[los['time'].isnull(),'all_walk'] = True
    try:
        los['time'] = los['time'].fillna(los["{'bus', 'rail'}"])
    except:
        los['time'] = los['time'].fillna(los["{'rail', 'bus'}"])
            
    return los

In [36]:
los = format_los(sm,volumes_pt)
los[['origin','destination','vehicle_class','time']].to_csv(output_folder +'pt_skim.csv')

# ref pathfinder

In [37]:
ref.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster!
)

start publicpathfinder
build_mode_combinations
find_broken_mode_paths


breaking modes: set() : 100%|█████████████████████| 4/4 [00:24<00:00,  6.15s/it]


path_analysis


In [38]:
ref_los = format_los(ref, volumes_pt)
ref_los[['origin','destination','vehicle_class','time']].to_csv(output_folder +'ref_pt_skim.csv')


In [39]:
#ref_los[~ref_los['all_walk']]

In [40]:
#sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

# Car pathfinder

In [41]:
volumes_car = volumes[volumes['vehicle_class'].isin(['C_other','C_EB'])]
od_set = set(zip(volumes_car['origin'], volumes_car['destination']))

In [42]:

car = sm.copy()
car.road_links['dup'] = car.road_links['a'] + car.road_links['b']
car.road_links = car.road_links.drop_duplicates('dup').drop(columns=['dup'])

car.preparation_ntlegs(
    short_leg_speed=15,
    long_leg_speed=15,
    threshold=5000,
    zone_to_transit=False,
    zone_to_road=True,
    n_ntlegs=2,
    max_ntleg_length=10000000000,
)
car.step_road_pathfinder(method='aon', access_time='time',od_set=od_set,path_analysis=True,)


self.volumes does not exist. od generated with self.zones


In [43]:
los = volumes_car[['origin','destination','vehicle_class']].copy()
los = los.drop_duplicates()

los = los.merge(car.car_los[['origin','destination','time']], on = ['origin','destination'])
los.loc[los['vehicle_class']=='C_EB','time'] = los.loc[los['vehicle_class']=='C_EB','time'] * 0.98

los[['origin','destination','vehicle_class','time']].to_csv(output_folder +'car_skim.csv')

then reference with time = road_times

In [44]:
car = ref.copy()

car.road_links['time']  = road_times['time']

car.road_links['dup'] = car.road_links['a'] + car.road_links['b']
car.road_links = car.road_links.drop_duplicates('dup').drop(columns=['dup'])

car.preparation_ntlegs(
    short_leg_speed=15,
    long_leg_speed=15,
    threshold=5000,
    zone_to_transit=False,
    zone_to_road=True,
    n_ntlegs=1,
    max_ntleg_length=10000000000,
)
car.step_road_pathfinder(method='aon', access_time='time',od_set=od_set)


self.volumes does not exist. od generated with self.zones


In [45]:
los = volumes_car[['origin','destination','vehicle_class']].copy()
los = los.drop_duplicates()

los = los.merge(car.car_los[['origin','destination','time']], on = ['origin','destination'])
los.loc[los['vehicle_class']=='C_EB','time'] = los.loc[los['vehicle_class']=='C_EB','time'] * 0.98

los[['origin','destination','vehicle_class','time']].to_csv(output_folder +'ref_car_skim.csv')

In [47]:
los

,origin,destination,vehicle_class,set(),{'bus'},"{'rail', 'bus'}",{'rail'}
0,zone_1,zone_1,BPAX,NaN,NaN,0.000000,NaN
1,zone_1,zone_6,BPAX,NaN,NaN,3662.452930,1991.236862
2,zone_1,zone_7,BPAX,NaN,NaN,3189.681990,2033.751067
3,zone_1,zone_10,BPAX,NaN,NaN,2930.800698,1236.471043
4,zone_1,zone_11,BPAX,NaN,NaN,2121.424850,982.150246
...,...,...,...,...,...,...,...
93986,zone_701,zone_695,RPAX,NaN,305503.205703,307464.920108,229773.493888
93987,zone_701,zone_696,RPAX,NaN,301616.387876,302761.347103,227556.497953
93988,zone_701,zone_697,RPAX,NaN,NaN,335555.326746,227798.142879
93989,zone_701,zone_698,RPAX,248908.246601,331253.852599,384201.875636,264972.416027
